### OpenCV + Caffe (res10 net) Face Detection Video ###

In [5]:
# ref: https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/

In [17]:
import numpy as np
import time
import cv2
import matplotlib.pyplot as plt

### Setup Functions ###

In [18]:
def net_processing(img, net):
    # Pass img/frame, net and output layers get output:

    start= time.time()

    # Modify img to input format:
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Pass to net:
    net.setInput(blob)
    detections = net.forward()

    end = time.time()
    
    process_time = end - start
    
    return detections, process_time

In [29]:
def net_detect(img, net, des_threshold):
    # Output format:
    # (pc, bx, by, bh, bw, .... class_preds)

    (H, W) = img.shape[:2]
    
    detections, process_time = net_processing(img, net)
    
    locations = np.zeros((5,))
    
    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]
 
        if confidence >= des_threshold:
        
            box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
            (startX, startY, endX, endY) = box.astype("int")
            
            box_info = [(i+1), startX, startY, endX, endY]
            locations = np.vstack([locations, box_info])
            
            (w, h) = ((endX - startX), (endY - startY))
    
            text = "Face{}: {:.2f}%".format((i+1), confidence * 100)
                
            # add litle background to class name info:
            backg = np.full((img.shape), (0,0,0), dtype=np.uint8)
            cv2.putText(backg, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
            fx,fy,fw,fh = cv2.boundingRect(backg[:,:,2])
                
            # Draw obj bbox:
            cv2.rectangle(img, (startX, startY), (startX + w, startY + h), (0, 0, 255), 2) 
            cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), (0, 0, 255), -1) 
            cv2.rectangle(img, (fx, fy), (fx + fw, fy + fh), (0, 0, 255), 3) 
            cv2.putText(img, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,0), 1)         
    
    return img, locations[1:], process_time

### Object Detection and Classification: ###

In [30]:
## Fast Caffe files from: https://github.com/vinuvish/Face-detection-with-OpenCV-and-deep-learning/tree/master/models

In [31]:
# Net Architecture Weights and Configuration:
caffe_proto = 'deploy.prototxt.txt'
caffe_model = 'res10_300x300_ssd_iter_140000.caffemodel'

net = cv2.dnn.readNetFromCaffe(caffe_proto, caffe_model)

In [32]:
# Define video file:
video = 'people.mp4'

# Define threshold:
des_threshold = 0.4

In [33]:
# Load Video:
cap = cv2.VideoCapture(video)
video_fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    
    start = time.time()
    
    success, frame = cap.read()
    
    if not success:
        print('Server OFF')
        break
        
    frame = cv2.resize(frame, (600,500))
    
    ###############################################################################################################
    
    frame, locations, process_time = net_detect(frame, net, des_threshold)
    
    ###############################################################################################################
    
    end = time.time()

    frame_time = (end - start) + 0.0001
    fps = np.floor(1/frame_time)
        
    if (fps > video_fps):
        time.sleep(1/video_fps)
        fps = video_fps
            
    cv2.putText(frame, "FPS: {}".format(fps), (30,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # Keyboard Controls:
    
    key = cv2.waitKey(1) or 0xff   
        
    if key == ord('k'):
        break
    
    cv2.imshow("Output", frame)
    
cv2.destroyAllWindows()
cap.release()

print('Caffe Process time:', process_time)
print('Last locations:', locations)

Caffe Process time: 0.06247305870056152
Last locations: [[  1. 147.  96. 190. 179.]
 [  2. 287. 100. 332. 178.]
 [  3.  65.  66. 112. 170.]
 [  4. 470.  84. 512. 180.]]
